TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.train import latest_checkpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
import random
import textwrap

In [2]:
# Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
#tag_name = 'all_transcripts'
#tag_name = 'brain'
#tag_name = 'business'
#tag_name = 'creativity'
#tag_name = 'culture'
#tag_name = 'psychology'
#tag_name = 'science'
tag_name = 'all'

lower_corpus = False

if lower_corpus:
    corpus_file_name = tag_name + '_lowercase'
else:
    corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
    data_path = f'{root_path}'
else:
    root_path = '../'
    data_path = f'{root_path}data/'
    
model_path = f'{root_path}models/'
logs_path = f'{root_path}logs/'
    
sequence_length = 80

In [4]:
with open(f'{data_path}{corpus_file_name}.txt', encoding='utf-8') as f:
    corpus = f.read()
f.close()

In [5]:
corpus[0:500]

"Good morning. How are you?  Good. It's been great, hasn't it? I've been blown away by the whole thing. In fact, I'm leaving.  There have been three themes running through the conference, which are relevant to what I want to talk about. One is the extraordinary evidence of human creativity in all of the presentations that we've had and in all of the people here; just the variety of it and the range of it. The second is that it's put us in a place where we have no idea what's going to happen in te"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [6]:
all_chars = sorted(list(set(corpus)))

#remove most punctuation, but leave spaces, exclamation points, and periods
list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—', '%','[', ']' ]

chars = [character for character in all_chars if character not in list_to_remove]
print(chars)
# hat tip to this post for the idea of removing one list from another
# https://www.geeksforgeeks.org/python-remove-all-values-from-a-list-present-in-other-list/

[' ', '!', '$', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'è', 'é', '–', '’', '“', '”', '…', '♫']


In [7]:
# remove all unwanted characters from the original corpus
corpus_list = [character for character in corpus if character not in list_to_remove]

# Re-join all letters back into a single string as the updated corpus
corpus = ''.join(corpus_list)

In [8]:
# If running models with punctuation removed, leave commented out
#chars = sorted(list(set(corpus)))

print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 75


In [9]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '$': 2, '.': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '?': 14, 'A': 15, 'B': 16, 'C': 17, 'D': 18, 'E': 19, 'F': 20, 'G': 21, 'H': 22, 'I': 23, 'J': 24, 'K': 25, 'L': 26, 'M': 27, 'N': 28, 'O': 29, 'P': 30, 'Q': 31, 'R': 32, 'S': 33, 'T': 34, 'U': 35, 'V': 36, 'W': 37, 'X': 38, 'Y': 39, 'Z': 40, 'a': 41, 'b': 42, 'c': 43, 'd': 44, 'e': 45, 'f': 46, 'g': 47, 'h': 48, 'i': 49, 'j': 50, 'k': 51, 'l': 52, 'm': 53, 'n': 54, 'o': 55, 'p': 56, 'q': 57, 'r': 58, 's': 59, 't': 60, 'u': 61, 'v': 62, 'w': 63, 'x': 64, 'y': 65, 'z': 66, 'è': 67, 'é': 68, '–': 69, '’': 70, '“': 71, '”': 72, '…': 73, '♫': 74}


In [10]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '$', 3: '.', 4: '0', 5: '1', 6: '2', 7: '3', 8: '4', 9: '5', 10: '6', 11: '7', 12: '8', 13: '9', 14: '?', 15: 'A', 16: 'B', 17: 'C', 18: 'D', 19: 'E', 20: 'F', 21: 'G', 22: 'H', 23: 'I', 24: 'J', 25: 'K', 26: 'L', 27: 'M', 28: 'N', 29: 'O', 30: 'P', 31: 'Q', 32: 'R', 33: 'S', 34: 'T', 35: 'U', 36: 'V', 37: 'W', 38: 'X', 39: 'Y', 40: 'Z', 41: 'a', 42: 'b', 43: 'c', 44: 'd', 45: 'e', 46: 'f', 47: 'g', 48: 'h', 49: 'i', 50: 'j', 51: 'k', 52: 'l', 53: 'm', 54: 'n', 55: 'o', 56: 'p', 57: 'q', 58: 'r', 59: 's', 60: 't', 61: 'u', 62: 'v', 63: 'w', 64: 'x', 65: 'y', 66: 'z', 67: 'è', 68: 'é', 69: '–', 70: '’', 71: '“', 72: '”', 73: '…', 74: '♫'}


In [11]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 267,106 characters.


In [23]:
#Save to csv to save computation time for streamlit web app
pd.DataFrame.from_dict(indices_char, orient='index').to_csv('../web/indices_char.csv')

In [ ]:
#Save to csv to save computation time for streamlit web app
pd.DataFrame.from_dict(char_indices, orient='index').to_csv('../web/char_indices.csv')

The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [454]:
corpus[0:50]

'Good morning. How are you?  Good. Its been great h'

In [455]:
corpus[1:41]

'ood morning. How are you?  Good. Its bee'

In [456]:
corpus[41]

'n'

In [25]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [26]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[21, 55, 55, 44, 0, 53, 55, 58, 54, 49, 54, 47, 3, 0, 22, 55, 63, 0, 41, 58, 45, 0, 65, 55, 61, 14, 0, 0, 21, 55, 55, 44, 3, 0, 23, 60, 59, 0, 42, 45, 45, 54, 0, 47, 58, 45, 41, 60, 0, 48, 41, 59, 54, 60, 0, 49, 60, 14, 0, 23, 62, 45, 0, 42, 45, 45, 54, 0, 42, 52, 55, 63, 54, 0, 41, 63, 41, 65, 0, 42]
[55, 55, 44, 0, 53, 55, 58, 54, 49, 54, 47, 3, 0, 22, 55, 63, 0, 41, 58, 45, 0, 65, 55, 61, 14, 0, 0, 21, 55, 55, 44, 3, 0, 23, 60, 59, 0, 42, 45, 45, 54, 0, 47, 58, 45, 41, 60, 0, 48, 41, 59, 54, 60, 0, 49, 60, 14, 0, 23, 62, 45, 0, 42, 45, 45, 54, 0, 42, 52, 55, 63, 54, 0, 41, 63, 41, 65, 0, 42, 65]


In [36]:
pd.DataFrame(X_numeric_list).to_csv('../web/X_numeric.csv', index=False)

In [37]:
pd.DataFrame(y_numeric_list).to_csv('../web/y_numeric.csv', index=False)

In [461]:
corpus[0:43]

'Good morning. How are you?  Good. Its been '

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [462]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 267,026 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [463]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [464]:
X.shape

(267026, 80, 1)

In [465]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [466]:
# Look at the first value of X
#X[0]

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [467]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

---
### Load LSTM Model
In this notebook, load a model that was previously trained and saved.

In [468]:
# Save the model to my Google Drive so I can load it later from another notebook
# model = load_model(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

# model.summary()

In [469]:
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))


# # Load the previously saved weights
# model.load_weights(latest)

# model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics = ['accuracy','Recall','Precision'])


#Models are saved in this format:
# model.save(f'{model_path}/{tag_name}_{input_nodes_count}_{layer_2_nodes_count}')
print(f'Loading model:{model_path}{tag_name}_50_75_80_seq')
model = load_model(f'{model_path}{tag_name}_500_75_80')

# print(f'Loading model:{model_path}{tag_name}_40_256')
# model = load_model(f'{model_path}{tag_name}_40_256')

model.summary()

Loading model:../models/all_50_75_80_seq
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 500)               1004000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 75)                37575     
Total params: 1,041,575
Trainable params: 1,041,575
Non-trainable params: 0
_________________________________________________________________


X_numeric_list contains all of our 40 character sequences. Choose one for a seed for the text generation.

In [470]:
seed_start = random.randint(0, len(corpus) - sequence_length)
print(f'Random start index for seed text: {seed_start} sequence.')

X_seeded = X_numeric_list[seed_start]
print(X_seeded)

Random start index for seed text: 146483 sequence.
[48, 49, 59, 0, 42, 55, 55, 51, 0, 26, 49, 45, 59, 56, 55, 60, 60, 49, 54, 47, 0, 54, 55, 0, 55, 54, 45, 0, 63, 41, 54, 60, 59, 0, 60, 55, 0, 53, 45, 45, 60, 0, 53, 45, 0, 49, 54, 0, 56, 45, 58, 59, 55, 54, 0, 41, 54, 65, 53, 55, 58, 45, 0, 54, 55, 0, 54, 55, 0, 54, 55, 0, 54, 55, 0, 54, 55, 3, 0, 34]


Convert this back to text so we can see what it looks like.

In [471]:
seed_string = ''.join([indices_char[letter_code] for letter_code in X_seeded])
seed_string

'his book Liespotting no one wants to meet me in person anymore no no no no no. T'

In [472]:
# We can also use the seed_start value to see a larger picture of the original text
for j in range(seed_start, seed_start + 15):
    seed_numeric = X_numeric_list[j]
    print (''.join([indices_char[letter_code] for letter_code in seed_numeric]))

his book Liespotting no one wants to meet me in person anymore no no no no no. T
is book Liespotting no one wants to meet me in person anymore no no no no no. Th
s book Liespotting no one wants to meet me in person anymore no no no no no. The
 book Liespotting no one wants to meet me in person anymore no no no no no. They
book Liespotting no one wants to meet me in person anymore no no no no no. They 
ook Liespotting no one wants to meet me in person anymore no no no no no. They s
ok Liespotting no one wants to meet me in person anymore no no no no no. They sa
k Liespotting no one wants to meet me in person anymore no no no no no. They say
 Liespotting no one wants to meet me in person anymore no no no no no. They say 
Liespotting no one wants to meet me in person anymore no no no no no. They say I
iespotting no one wants to meet me in person anymore no no no no no. They say It
espotting no one wants to meet me in person anymore no no no no no. They say Its
spotting no one wants to mee

To generate text, we need to change the input text into a 3D shape as we did the X input data.

In [473]:
temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
# normalize/scale the data by dividing by the length of the character list.
temp_x = temp_x / len(chars)
prediction = model.predict(temp_x, verbose=0)
print(f"A prediction's shape:{prediction.shape}")
print('Prediction:')
print (prediction)

A prediction's shape:(1, 75)
Prediction:
[[7.62266591e-02 1.56492236e-10 7.49858009e-10 1.60318259e-02
  2.73117053e-07 2.83879245e-07 1.86448960e-06 1.35662344e-06
  8.02136037e-06 7.71028681e-06 5.10848244e-04 1.34363449e-06
  7.32487127e-11 1.14947204e-06 3.36395529e-11 3.70571379e-10
  4.99591416e-13 1.01771785e-11 5.68774333e-11 2.73902554e-08
  3.22840227e-11 2.09589412e-08 2.71267600e-12 1.09397511e-07
  1.04868675e-11 4.04120425e-16 5.08398026e-19 7.02022399e-11
  1.31574534e-13 8.80105655e-10 2.48859716e-13 1.91020651e-14
  8.70046602e-13 4.33746178e-10 9.45029939e-08 7.68499500e-12
  4.78442537e-16 8.30269176e-13 5.84251436e-15 5.64952443e-14
  1.45062042e-16 3.29776853e-02 3.70638096e-03 1.77909285e-02
  4.50406671e-02 1.18002005e-01 8.07497092e-03 4.96369693e-03
  1.54471353e-01 3.97264697e-02 8.50018637e-07 1.48961935e-02
  1.97547134e-02 4.17924523e-02 3.48653160e-02 1.53230965e-01
  1.04755405e-02 2.48772845e-13 3.75947393e-02 3.18075679e-02
  5.99242896e-02 2.02435795e-

Each prediction produces an result of shape (1, 75), and contains a predicted probability of a character. Use argmax to find the index of the highest probability.

In [474]:
np.argmax(prediction)

48

So the first letter predicted after the seed sequence is:

In [475]:
indices_char[np.argmax(prediction)]

'h'

In [476]:
# Put all predicted characters in a list, then convert to a string later.
complete_predicted_text =[]

# Generate 1000 characters
for i in range(600):
    # reshape to 1 row, sequence length, and 1 for predicting one character as the target
    temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
    # normalize/scale the data by dividing by the length of the character list.
    temp_x = temp_x / len(chars)
    prediction = model.predict(temp_x, verbose=0)
    complete_predicted_text.append(indices_char[np.argmax(prediction)])

    # After predicting, add this character's numeric value on to the randomly
    # chose seed string, and use this for another prediction
    X_seeded.append(np.argmax(prediction))

    # Move the seeded text up one character, so create a new sequence
    X_seeded = X_seeded[1:len(X_seeded)]


#print (complete_predicted_text)


The predicted results are a list of characters, so convert into a string

In [477]:
predicted_string = ''.join(complete_predicted_text)

# Wrap to 120 characters just so I can visually compare to the original string.
textwrap.wrap(seed_string + '->'+ predicted_string, width=120)

['his book Liespotting no one wants to meet me in person anymore no no no no no. T->h th toe to the toe to toe th the  Se',
 'the the th te tee toe axd I woe thu to the to the wo to the to th to I  5T Iq t to the thx to th to te t te the toete',
 'In toe to qoe to t ae to toe to th F Od to te wh th th to te toe th to a teoe toe to  I2 a  I wo te I nan  02n jo the to',
 'th toe wo to wo N wo I no toe thet   Hh to to to to to woe to th th the toe th I woe th Iod I ao tou th tee th th to toe',
 'tee th to to toe to the toe tee the toe toe  An th th  Vh whe to te the th th te to toe th to we th to the  To th toe to',
 'I bo th the D wou to te to  In whx te te the  So I se th I we to th to th the to']

In [478]:
# Find the original seed string to see what came after it
original_paragraph = corpus[corpus.find(seed_string): corpus.find(seed_string) + 2000]

textwrap.wrap(original_paragraph, width=120)

['his book Liespotting no one wants to meet me in person anymore no no no no no. They say Its okay well email you.  I cant',
 'even get a coffee date at Starbucks. My husbands like Honey deception? Maybe you could have focused on cooking. How',
 'about French cooking? So before I get started what Im going to do is Im going to clarify my goal for you which is not to',
 'teach a game of Gotcha. Liespotters arent those nitpicky kids those kids in the back of the room that are shouting',
 'Gotcha! Gotcha! Your eyebrow twitched. You flared your nostril. I watch that TV show Lie To Me. I know youre lying. No',
 'liespotters are armed with scientific knowledge of how to spot deception. They use it to get to the truth and they do',
 'what mature leaders do everyday they have difficult conversations with difficult people sometimes during very difficult',
 'times. And they start up that path by accepting a core proposition and that proposition is the  Lying is a cooperative',
 'act. Think about 